<a href="https://colab.research.google.com/github/LeoProvorov/Data-science/blob/master/Country_and_DataScience_with_ALTAIR_visualisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# download kaggle

In [1]:
from google.colab import files

files.upload() # Загрузка архива с кегл. Закидываем файл json сюда

# Расположение файла для аутентификации на kaggle.com и получение прав доступа
!mkdir -p ~/.kaggle
!mv -i kaggle.json ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

# загрузка данных
# !kaggle competitions download -c titanic 
# or !kaggle datasets download -d heptapod/titanic


!ls

Saving kaggle.json to kaggle.json
sample_data


In [2]:
# загрузка данных
!kaggle competitions download -c kaggle-survey-2020

  0% 0.00/46.5k [00:00<?, ?B/s]
100% 46.5k/46.5k [00:00<00:00, 18.3MB/s]
  0% 0.00/111k [00:00<?, ?B/s]
100% 111k/111k [00:00<00:00, 35.4MB/s]
  0% 0.00/1.94M [00:00<?, ?B/s]
100% 1.94M/1.94M [00:00<00:00, 63.5MB/s]


# Обзор состояния науки о данных Kaggle 2020

Kaggle - мировой лидер в области конкурентной науки о данных. С 2017 года он проводит ежегодное исследование состояния науки о данных. Этот блокнот проведет EDA данных опроса 2020 года. В центре внимания этого EDA будет то, как Data Science различается в зависимости от страны. 

In [13]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import altair as alt
#enable altair to plot all the data
alt.data_transformers.disable_max_rows()
alt.themes.enable('fivethirtyeight')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [14]:
# Load the data
df_survey_responses = pd.read_csv("kaggle_survey_2020_responses.csv.zip")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [15]:
col_list = df_survey_responses.columns.to_list()

#create a mapping dict from question id to question content
question_mapping = df_survey_responses.loc[0].to_dict()

In [17]:
df_survey_responses = df_survey_responses.drop(0, axis=0)
#shorten longest country names
df_survey_responses.loc[df_survey_responses['Q3']=="United Kingdom of Great Britain and Northern Ireland", "Q3"] = "United Kingdom"
df_survey_responses.loc[df_survey_responses['Q3']=="Iran, Islamic Republic of...", "Q3"] = "Iran"
df_survey_responses.head(3)

,Time from Start to Finish (seconds),Q1,Q2,Q3,Q4,Q5,Q6,Q7_Part_1,Q7_Part_2,Q7_Part_3,Q7_Part_4,Q7_Part_5,Q7_Part_6,Q7_Part_7,Q7_Part_8,Q7_Part_9,Q7_Part_10,Q7_Part_11,Q7_Part_12,Q7_OTHER,Q8,Q9_Part_1,Q9_Part_2,Q9_Part_3,Q9_Part_4,Q9_Part_5,Q9_Part_6,Q9_Part_7,Q9_Part_8,Q9_Part_9,Q9_Part_10,Q9_Part_11,Q9_OTHER,Q10_Part_1,Q10_Part_2,Q10_Part_3,Q10_Part_4,Q10_Part_5,Q10_Part_6,Q10_Part_7,...,Q31_B_Part_7,Q31_B_Part_8,Q31_B_Part_9,Q31_B_Part_10,Q31_B_Part_11,Q31_B_Part_12,Q31_B_Part_13,Q31_B_Part_14,Q31_B_OTHER,Q33_B_Part_1,Q33_B_Part_2,Q33_B_Part_3,Q33_B_Part_4,Q33_B_Part_5,Q33_B_Part_6,Q33_B_Part_7,Q33_B_OTHER,Q34_B_Part_1,Q34_B_Part_2,Q34_B_Part_3,Q34_B_Part_4,Q34_B_Part_5,Q34_B_Part_6,Q34_B_Part_7,Q34_B_Part_8,Q34_B_Part_9,Q34_B_Part_10,Q34_B_Part_11,Q34_B_OTHER,Q35_B_Part_1,Q35_B_Part_2,Q35_B_Part_3,Q35_B_Part_4,Q35_B_Part_5,Q35_B_Part_6,Q35_B_Part_7,Q35_B_Part_8,Q35_B_Part_9,Q35_B_Part_10,Q35_B_OTHER
1,1838,35-39,Man,Colombia,Doctoral degree,Student,5-10 years,Python,R,SQL,C,NaN,NaN,Javascript,NaN,NaN,NaN,MATLAB,NaN,Other,Python,"Jupyter (JupyterLab, Jupyter Notebooks, etc)",NaN,NaN,Visual Studio Code (VSCode),NaN,Spyder,NaN,NaN,NaN,NaN,NaN,NaN,Kaggle Notebooks,Colab Notebooks,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,SAP Analytics Cloud,NaN,NaN,"Automated data augmentation (e.g. imgaug, albu...",NaN,NaN,NaN,Automated hyperparameter tuning (e.g. hyperopt...,Automation of full ML pipelines (e.g. Google C...,NaN,NaN,Google Cloud AutoML,NaN,Databricks AutoML,NaN,NaN,Auto-Keras,Auto-Sklearn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TensorBoard,NaN,NaN,NaN,NaN,NaN,NaN
2,289287,30-34,Man,United States of America,Master’s degree,Data Engineer,5-10 years,Python,R,SQL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Python,NaN,NaN,Visual Studio,NaN,PyCharm,NaN,NaN,Sublime Text,NaN,NaN,NaN,NaN,NaN,Colab Notebooks,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,860,35-39,Man,Argentina,Bachelor’s degree,Software Engineer,10-20 years,NaN,NaN,NaN,NaN,NaN,Java,Javascript,NaN,NaN,Bash,NaN,NaN,NaN,R,NaN,NaN,NaN,Visual Studio Code (VSCode),NaN,NaN,Notepad++,Sublime Text,Vim / Emacs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN


In [85]:
# count country data

country = df_survey_responses.groupby(['Q3'])
df_country = country.agg('count')
df_country.reset_index(inplace=True)
df_country=df_country[['Q3', 'Q1']]
df_country.columns=('Country', 'Count')
df_country.tail()

,Country,Count
50,Ukraine,170
51,United Arab Emirates,59
52,United Kingdom,489
53,United States of America,2237
54,Viet Nam,147


In [86]:
def chart_country(df, sort=None):
    chart_country_groups = alt.Chart(df_country).mark_bar().encode(
    alt.X('Country:N', sort=sort, axis=alt.Axis(grid=False)),
    alt.Y('Count:Q', title="Count of Responses", axis=alt.Axis(grid=False))
    ).properties(width=680,
    background='#f5f5f5',title={
"text": ["Number of Respondents per Country"],
"fontSize":20, "fontWeight":600, "subtitleFontSize":14
}
    )
    
    return chart_country_groups

chart_country_groups = chart_country(df_country)


In [87]:
#countries sorted by count
df_country = df_country.sort_values(by=['Count'], ascending=False)
country_sort = df_country['Country'].to_list()
chart_country_groups = chart_country(df_country)

chart_country_groups

alt.Chart(...)

Здесь мы можем увидеть ответы по странам. Индия с большим отрывом опережает США. На третьем месте находится «Прочие», то есть все остальные страны, не включенные в список, сгруппированные в одну категорию. Давайте углубимся в данные.

# Разбивка по странам и возрасту
Мы разбиваем ответы по возрасту и стране и подсчитываем количество записей в каждом. Мы регистрируем преобразование значений счетчика, чтобы получить более информативную визуализацию.

In [92]:
age_country = df_survey_responses.groupby(['Q1', 'Q3'])
df_age_country = age_country.agg('count')
df_age_country.reset_index(inplace=True)
df_age_country=df_age_country[['Q1', 'Q3', 'Q2']]
df_age_country.columns=('Age', 'Country', 'Count')
df_age_country['Count'] = np.log(df_age_country['Count'])

#sort by count
df_age_country = df_age_country.sort_values(by=['Count'], ascending=False)

df_age_country.head()


,Age,Country,Count
15,18-21,India,7.629976
68,22-24,India,7.324490
123,25-29,India,6.962243
178,30-34,India,6.276643
216,30-34,United States of America,5.986452


In [96]:
alt.Chart(df_age_country).mark_rect().encode(
    alt.X('Country:N', sort=None, title="Country sorted by descending count of aggregate responses"),
    alt.Y('Age:N'),
    color=alt.Color('Count:Q', legend=alt.Legend(title="Log Count of Respondents", orient="top"))
).properties(width=680,
    background='#f5f5f5',title={
"text": ["Age of Respondents"],
"subtitle": ["Broken Down by proportionally by Country"],
"fontSize":20, "fontWeight":600, "subtitleFontSize":14
}
)

alt.Chart(...)

Мы видим, что в Индии многие респонденты относятся к самым молодым возрастным категориям. Сравнивая это с другими странами, мы видим, что респонденты в других странах, как правило, старше. Особенно это заметно в США.

Примечание об образцах данных 
Хотя в этой записной книжке основное внимание уделяется анализу данных по странам, на следующей диаграмме видно, что возраст респондентов влияет на другие аспекты данных, в данном случае на то, сколько времени потребовалось для заполнения опроса в секундах (максимум 3000 секунд или 50 минут). Мы должны помнить, просматривая более поздние диаграммы, и понимать, что они не выбираются случайным образом и, как следствие, могут иметь смещение выборки. Если мы подготовим ту же диаграмму по странам, то может показаться, что респонденты из Индии были быстрее, но это может ничего не сказать об Индии. Возможно, причина в том, что респонденты из Индии, как правило, моложе.

In [99]:
# large violinplots in Altair can look good but can be computationally intensive and it may be better to consider using matplotlib/seaborn
# altair usually limits input sources to 5000 rows but I have overriden that setting
# the code commented below produces the chart below, but the chart is loaded from a saved image that uses less memory

# alt.Chart(df_survey_responses).transform_density(
#     'Time from Start to Finish (seconds)',
#     as_=['Time from Start to Finish (seconds)', 'density'],
#     extent=[0, 3000],
#     groupby=['Q1']
# ).mark_area(orient='horizontal').encode(
#     y=alt.Y('Time from Start to Finish (seconds):Q', title=''),
#     color=alt.Color('Q1:N', legend=alt.Legend(title="Age", orient="top")),
#     x=alt.X(
#         'density:Q',
#         stack='center',
#         impute=None,
#         title=None,
#         axis=alt.Axis(labels=False, values=[0],grid=False, ticks=True),
#     ),
#     column=alt.Column(
#         'Q1:N',
#         header=alt.Header(
#             titleOrient='bottom',
#             labelOrient='bottom',
#             labelPadding=0,
#         ),
#     )
# ).properties(width=80,
#     background='#f5f5f5',title={
# "text": ["Survey Response Time in Seconds"],
# "subtitle": ["Broken down by Age"],
# "fontSize":20, "fontWeight":600, "subtitleFontSize":14
# }
# ).configure_facet(
#     spacing=0
# ).configure_view(
#     stroke=None
# )

Мы сможем более четко увидеть различный возраст респондентов в разных странах, если сосредоточимся на восьми странах с наибольшим числом респондентов.

In [100]:
#only top countries

alt.Chart(df_age_country[df_age_country['Country'].isin(country_sort[:9])]).mark_bar(
    cornerRadiusTopLeft=3,
    cornerRadiusTopRight=3
).encode(
    x=alt.X('Count:Q', stack="normalize", title="Normalized Count"),
    y=alt.Y('Age:O'),
    color=alt.Color('Country:N', legend=alt.Legend(orient="right"))
).properties(width=500,
             background='#f5f5f5',title={
"text": ["Age of Respondents"],
"subtitle": ["Broken Down by proportionally by Country"],
"fontSize":20, "fontWeight":600, "subtitleFontSize":14
})

alt.Chart(...)

Разбивка по странам и высшим достижениям в образовании
 Давайте посмотрим на распределение наивысших образовательных достижений по странам в этом опросе Kaggle. Помните, что в этих результатах может быть ошибка отбора, особенно если принять во внимание возраст респондентов. Например, респонденты в возрасте 18-21 лет вряд ли будут иметь докторскую степень! Если в этой возрастной группе есть респонденты с докторской степенью, пожалуйста, свяжитесь со мной по поводу объединения в будущих соревнованиях ;-)

In [101]:
education_country = df_survey_responses.groupby(['Q4', 'Q3'])
df_education_country = education_country.agg('count')
df_education_country.reset_index(inplace=True)
df_education_country=df_education_country[['Q4', 'Q3', 'Q2']]
df_education_country.columns=('education', 'country', 'Count')
df_education_country['rank'] = df_education_country['education']
education_sort_map = {"Doctoral degree": 6, 'Master’s degree': 5, "Professional degree": 4, 'Bachelor’s degree': 3, 
                  "Some college/university study without earning a bachelor’s degree": 2,
                  "No formal education past high school": 1, "I prefer not to answer": 0
                 }
df_education_country['rank'] = df_education_country['rank'].replace(education_sort_map)
df_education_country['education_rank'] = df_education_country['rank'].astype(str) + ' - ' + df_education_country['education']

In [102]:
alt.Chart(df_education_country).mark_bar(
    cornerRadiusTopLeft=3,
    cornerRadiusTopRight=3
).encode(
    x=alt.X('country:N'),
    y=alt.Y('Count:Q', stack="normalize", title="Normalized Count"),
    color=alt.Color('education_rank:N', legend=alt.Legend(orient="left"))
).properties(width=520,
             background='#f5f5f5',title={
"text": ["Highest Educational Achievement Held by Respondents"],
"subtitle": ["Broken Down by proportionally by Country"],
"fontSize":18, "fontWeight":600, "subtitleFontSize":14
})

alt.Chart(...)

Разбивка по странам и должностям Давайте посмотрим на распределение должностей по странам.

In [103]:
job_country = df_survey_responses.groupby(['Q5', 'Q3'])
df_job_country = job_country.agg('count')
df_job_country.reset_index(inplace=True)
df_job_country=df_job_country[['Q5', 'Q3', 'Q2']]
df_job_country.columns=('job', 'country', 'Count')

In [104]:
alt.Chart(df_job_country).mark_bar(
    cornerRadiusTopLeft=3,
    cornerRadiusTopRight=3
).encode(
    x=alt.X('country:N'),
    y=alt.Y('Count:Q', stack="normalize", title="Normalized Count"),
    color=alt.Color('job:N', legend=alt.Legend(orient="left"))
).properties(width=550,
             background='#f5f5f5',title={
"text": ["Job Titles Held by Respondents"],
"subtitle": ["Broken Down by proportionally by Country"],
"fontSize":18, "fontWeight":600, "subtitleFontSize":14
})

alt.Chart(...)

Мы видим, что в некоторых странах примерно половина респондентов - студенты. Давайте просто сосредоточимся на работе «Data Scientist».

In [105]:
df_job_proportions = df_job_country.merge(df_country, left_on='country', right_on='Country',
          suffixes=(None, '_total'))
df_job_proportions.drop(columns=['country'], inplace=True)
df_job_proportions['job_proportion'] = df_job_proportions['Count'] / df_job_proportions['Count_total']
df_job_proportions

,job,Count,Country,Count_total,job_proportion
0,Business Analyst,11,Argentina,134,0.082090
1,Currently not employed,7,Argentina,134,0.052239
2,DBA/Database Engineer,2,Argentina,134,0.014925
3,Data Analyst,15,Argentina,134,0.111940
4,Data Engineer,3,Argentina,134,0.022388
...,...,...,...,...,...
672,Other,5,Romania,61,0.081967
673,Research Scientist,4,Romania,61,0.065574
674,Software Engineer,15,Romania,61,0.245902
675,Statistician,1,Romania,61,0.016393


In [106]:
alt.Chart(df_job_proportions).mark_bar(color="grey").encode(
    alt.Y('Country:N', axis=alt.Axis(grid=False)),
    alt.X('job_proportion:Q', title="Proportion of Respondents", axis=alt.Axis(grid=False))
    ).properties(width=680, height=440,
    background='#f5f5f5',title={
"text": ["Proportion of Respondents who are Data Scientists"],
"subtitle": ["Broken Down by Country"],
"fontSize":20, "fontWeight":600, "subtitleFontSize":14
})

alt.Chart(...)

Разбивка по годам опыта программирования

In [107]:
experience_country = df_survey_responses.groupby(['Q6', 'Q3'])
df_experience_country = experience_country.agg('count')
df_experience_country.reset_index(inplace=True)
df_experience_country=df_experience_country[['Q6', 'Q3', 'Q2']]
df_experience_country.columns=('experience', 'country', 'Count')

df_experience_country['rank'] = df_experience_country['experience']
experience_sort_map = {"20+ years": 6, '10-20 years': 5, "5-10 years": 4, '3-5 years': 3, 
                  "1-2 years": 2,
                  "< 1 years": 1, "I have never written code": 0
                 }
df_experience_country['rank'] = df_experience_country['rank'].replace(experience_sort_map)
df_experience_country['experience_rank'] = df_experience_country['rank'].astype(str) + ' - ' + df_experience_country['experience']
df_experience_country

,experience,country,Count,rank,experience_rank
0,1-2 years,Argentina,21,2,2 - 1-2 years
1,1-2 years,Australia,39,2,2 - 1-2 years
2,1-2 years,Bangladesh,39,2,2 - 1-2 years
3,1-2 years,Belarus,12,2,2 - 1-2 years
4,1-2 years,Belgium,14,2,2 - 1-2 years
...,...,...,...,...,...
374,I have never written code,Ukraine,6,0,0 - I have never written code
375,I have never written code,United Arab Emirates,7,0,0 - I have never written code
376,I have never written code,United Kingdom,20,0,0 - I have never written code
377,I have never written code,United States of America,85,0,0 - I have never written code


In [108]:
alt.Chart(df_experience_country).mark_bar(
    cornerRadiusTopLeft=3,
    cornerRadiusTopRight=3
).encode(
    y=alt.Y('country:N', axis=alt.Axis(grid=False, titleAngle=0)),
    x=alt.X('Count:Q', stack="normalize", title="Normalized Count"),
    color=alt.Color('experience_rank:N', legend=alt.Legend(orient="bottom"), title="Years of Experience")
).properties(width=550, height=500,
             background='#f5f5f5',title={
"text": ["Years of Experience"],
"subtitle": ["Broken Down by proportionally by Country"],
"fontSize":18, "fontWeight":600, "subtitleFontSize":14
})

alt.Chart(...)

Какой язык вы бы посоветовали выучить в первую очередь начинающему специалисту по данным? Это дает представление о том, куда, по мнению Data Scientist, движется Data Science, и не омрачено наследием языков, которые Data Scientist могут знать или не знать. Опять же, следуя нашей теме, мы рассмотрим это по странам.

In [109]:
prog_lang_country = df_survey_responses.groupby(['Q8', 'Q3'])
df_prog_lang_country = prog_lang_country.agg('count')
df_prog_lang_country.reset_index(inplace=True)
df_prog_lang_country=df_prog_lang_country[['Q8', 'Q3', 'Q2']]
df_prog_lang_country.columns=('prog_lang', 'country', 'Count')
#sort by count
df_prog_lang_country = df_prog_lang_country.sort_values(by=['Count'], ascending=False)
df_prog_lang_country.head()

,prog_lang,country,Count
339,Python,India,4347
377,Python,United States of America,1531
353,Python,Other,880
329,Python,Brazil,493
345,Python,Japan,471


In [112]:
def prog_lang_country(df, title):
    chart_prog_lang_country = alt.Chart(df).mark_bar(
        cornerRadiusTopLeft=3,
        cornerRadiusTopRight=3
    ).encode(
        y=alt.Y('country:N', axis=alt.Axis(grid=False, titleAngle=0)),
        x=alt.X('Count:Q', stack="normalize", title="Normalized Count"),
        color=alt.Color('prog_lang:N', legend=alt.Legend(orient="bottom"), title="Programming Language")
    ).properties(width=550, height=500,
                 background='#f5f5f5',title={
    "text": [title],
    "subtitle": ["Broken Down by proportionally by Country"],
    "fontSize":18, "fontWeight":600, "subtitleFontSize":14
    })
      
    return chart_prog_lang_country
    

In [113]:
prog_lang_country(df_prog_lang_country, "Recommended Starting Programming Language for Data Science")

alt.Chart(...)

Мы видим, что Python на сегодняшний день является самым популярным языком, рекомендованным во всех странах. Давайте попробуем исключить python из списка, чтобы получить более четкое представление о том, какие другие языки может изучить интересный специалист по данным.

In [114]:
prog_lang_country(df_prog_lang_country.loc[df_prog_lang_country['prog_lang']!="Python"], "Recommended Starting Programming Language for Data Science (excluding Python)")

alt.Chart(...)

Возможно, неудивительно, что мы видим, что SQL и R широко рекомендуются во всех странах. Начинающие исследователи данных могут найти более подробную информацию о навыках, востребованных в их собственной стране, но помните, что размер выборки ответов для многих из перечисленных стран может быть не очень большим, и в результате результаты могут быть не очень надежными.

Какой тип вычислительной платформы вы чаще всего используете для своих проектов в области науки о данных? В вопросе 11 респондентов спрашивали, какую платформу они чаще всего используют для своих проектов в области науки о данных. Давайте посмотрим, как это разбивается по странам.

In [115]:
platform_country = df_survey_responses.groupby(['Q11', 'Q3'])
df_platform_country = platform_country.agg('count')
df_platform_country.reset_index(inplace=True)
df_platform_country=df_platform_country[['Q11', 'Q3', 'Q2']]
df_platform_country.columns=('platform', 'country', 'Count')
#sort by count
df_platform_country = df_platform_country.sort_values(by=['Count'], ascending=False)
df_platform_country.head()

,platform,country,Count
123,A personal computer or laptop,India,3935
161,A personal computer or laptop,United States of America,1497
137,A personal computer or laptop,Other,917
15,"A cloud computing platform (AWS, Azure, GCP, h...",India,658
113,A personal computer or laptop,Brazil,475


In [116]:
def platform_country_chart(df, title):
    chart_platform_country = alt.Chart(df).mark_bar(
        cornerRadiusTopLeft=3,
        cornerRadiusTopRight=3
    ).encode(
        y=alt.Y('country:N', axis=alt.Axis(grid=False, titleAngle=0)),
        x=alt.X('Count:Q', stack="normalize", title="Normalized Count"),
        color=alt.Color('platform:N', legend=alt.Legend(orient="right"), title="Platform")
    ).properties(width=500, height=500,
                 background='#f5f5f5',title={
    "text": [title],
    "subtitle": ["Broken Down by proportionally by Country"],
    "fontSize":18, "fontWeight":600, "subtitleFontSize":14
    })
    
    return chart_platform_country

In [117]:
alt.themes.enable('vox')
platform_country_chart(df_platform_country, "Preferred Platform for Data Science")

alt.Chart(...)

Мы снова видим, что есть одна категория, которая доминирует в каждой стране. Несмотря на рост облачных решений и глубокого обучения, персональный компьютер или ноутбук по-прежнему остается самым популярным выбором. Давайте посмотрим на другие категории, за исключением персональных компьютеров.

In [118]:
df_platform_country = df_platform_country.loc[df_platform_country['platform']!="A personal computer or laptop"]
alt.themes.enable('vox')
platform_country_chart(df_platform_country, "Preferred Platform for Data Science (excluding pc's)")

alt.Chart(...)

Использование TPU Остановившись на аппаратном обеспечении, посмотрим, сколько раз респонденты использовали TPU. (TPU - это специализированное оборудование, которое обеспечивает быструю обработку нейронных сетей.)

In [119]:
tpu_country = df_survey_responses.groupby(['Q13', 'Q3'])
df_tpu_country = tpu_country.agg('count')
df_tpu_country.reset_index(inplace=True)
df_tpu_country=df_tpu_country[['Q13', 'Q3', 'Q2']]
df_tpu_country.columns=('tpu_usage', 'country', 'Count')
#sort by count
df_tpu_country = df_tpu_country.sort_values(by=['Count'], ascending=False)
df_tpu_country.head()

df_tpu_country['rank'] = df_tpu_country['tpu_usage']
tpu_sort_map = {"More than 25 times": 0, '6-25 times': 1, 
                  "2-5 times": 2,
                  "Once": 3, "Never": 4
                 }
df_tpu_country['rank'] = df_tpu_country['rank'].replace(tpu_sort_map)
df_tpu_country['tpu_usage_rank'] = df_tpu_country['rank'].astype(str) + ' - ' + df_tpu_country['tpu_usage']


In [120]:
alt.themes.enable('latimes')
bars = alt.Chart(df_tpu_country).mark_bar(
        cornerRadiusTopLeft=3,
        cornerRadiusTopRight=3
    ).encode(
        y=alt.Y('country:N', axis=alt.Axis(grid=False, titleAngle=0)),
        x=alt.X('Count:Q', stack="normalize", title="Normalized Count"),
        color=alt.Color('tpu_usage_rank:N', legend=alt.Legend(orient="right"), title="TPU Usage")
    ).properties(width=500, height=500,title={
    "text": ["TPU Usage"],
    "subtitle": ["Broken Down by proportionally by Country"],
    "fontSize":18, "fontWeight":600, "subtitleFontSize":14
    })

bars


alt.Chart(...)

Во всех странах самый популярный ответ «Никогда» показывает, что большинство респондентов не использовали ТПУ. Ведущими странами по использованию TPU являются Вьетнам, Перу и Румыния, тогда как использование TPU реже всего в Ирландии. Еще раз мы должны рассмотреть вопросы размера выборки и систематической ошибки отбора, делая слишком много выводов из нашего набора данных.

Заключение В 2020 году Data Science процветает. В опросе Kaggle мы видим респондентов из более чем 50 разных стран. Работая с данными в опросе с разбивкой информации по странам, мы обнаружили много интересных идей. Например, респонденты из Индии, как правило, были моложе респондентов из США. Респонденты из Ирана и Швейцарии чаще имели докторскую степень. Респонденты из Швеции, скорее всего, имеют опыт программирования более 20 лет. Платформы глубокого обучения более популярны, чем облачные платформы в Бельгии и Вьетнаме. Я надеюсь, что этот блокнот был интересным и содержательным, и что вы узнали что-то новое о состоянии науки о данных в вашей стране.